In [1]:
import pandas as pd
import pickle

In [2]:
'''
# Set Pandas display options to show all rows and columns
pd.set_option("display.max_rows", None)  # Show all rows
pd.set_option("display.max_columns", None)  # Show all columns
pd.set_option("display.width", 1000)  # Adjust the width to accommodate more columns
'''

'\n# Set Pandas display options to show all rows and columns\npd.set_option("display.max_rows", None)  # Show all rows\npd.set_option("display.max_columns", None)  # Show all columns\npd.set_option("display.width", 1000)  # Adjust the width to accommodate more columns\n'

In [3]:
# Load the CSV file
csv_path = "candidates_identification.csv"
df = pd.read_csv(csv_path)

In [4]:
df.head()

,smiles
0,O=C1NC(C2=C1NCCN2)=O
1,O=C1NC(C2=C1NC3=C(C(NC3=O)=O)N2)=O
2,O=C1N(N2C(C(C)=C(C)C2=O)=O)C(C(C)=C1C)=O
3,O=C1N(C(C=C1)=O)N2C(C=CC2=O)=O
4,O=C1N(C(C(C(O[Li])=O)=C1C(O[Li])=O)=O)N2C(C(OC...


In [5]:
import pandas as pd
import selfies as sf
from rdkit import Chem
from rdkit.Chem import MolToSmiles
from rdkit import RDLogger
from sklearn.base import BaseEstimator, TransformerMixin

# Suppress RDKit warnings
RDLogger.DisableLog('rdApp.error')

class SMILESToSELFIESTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, pad_to_len=None):
        self.pad_to_len = pad_to_len
        self.symbol_to_idx = None
        self.idx_to_symbol = None

    def sanitize_smiles(self, smiles):
        try:
            # Replace '.' with 'X'
            smiles = smiles.replace('.', 'X')
            mol = Chem.MolFromSmiles(smiles, sanitize=True)
            if mol is not None:
                return MolToSmiles(mol, canonical=True)
            else:
                return None
        except:
            return None

    def smiles_to_selfies(self, smiles):
        sanitized_smiles = self.sanitize_smiles(smiles)
        if sanitized_smiles:
            try:
                return sf.encoder(sanitized_smiles)
            except sf.EncoderError:
                return None
        return None

    def build_vocabulary(self, selfies_list):
        alphabet = sf.get_alphabet_from_selfies(selfies_list)
        alphabet.add("[nop]")  # [nop] is a special padding symbol
        self.alphabet = list(sorted(alphabet))
        self.symbol_to_idx = {s: i for i, s in enumerate(self.alphabet)}
        self.idx_to_symbol = {i: s for i, s in enumerate(self.alphabet)}

    def selfies_to_numerical(self, selfies):
        if not self.symbol_to_idx:
            raise ValueError("Vocabulary is not built. Call build_vocabulary first.")
        label, one_hot = sf.selfies_to_encoding(
            selfies=selfies,
            vocab_stoi=self.symbol_to_idx,
            pad_to_len=self.pad_to_len,
            enc_type="both"
        )
        return label

    def print_alphabet_list(self):
        if self.alphabet:
            print("Alphabet list:", self.alphabet)
        else:
            print("Alphabet list has not been built yet.")
    
    def fit(self, X, y=None):
        X = X.copy()
        # Convert SMILES to SELFIES
        X['selfies'] = X['smiles'].apply(self.smiles_to_selfies)
        
        # Filter out rows with invalid SELFIES
        X = X.dropna(subset=['selfies'])

        # Build vocabulary
        selfies_list = X['selfies'].tolist()
        self.build_vocabulary(selfies_list)

        # Determine pad_to_len if not specified
        if self.pad_to_len is None:
            self.pad_to_len = max(sf.len_selfies(s) for s in selfies_list)
        
        return self

    def transform(self, X, y=None):
        X = X.copy()
        # Convert SMILES to SELFIES (if not already done in fit)
        if 'selfies' not in X.columns:
            X['selfies'] = X['smiles'].apply(self.smiles_to_selfies)
        
        # Filter out rows with invalid SELFIES
        X = X.dropna(subset=['selfies'])

        # Convert SELFIES to numerical IDs
        X['numerical_ids'] = X['selfies'].apply(self.selfies_to_numerical)
        
        #return X[['numerical_ids']]

        # Convert the list of numerical IDs into a DataFrame
        numerical_ids_df = pd.DataFrame(X['numerical_ids'].tolist())
        
        return numerical_ids_df

    def fit_transform(self, X, y=None):
        return self.fit(X).transform(X)


In [6]:
# Load the transformer state
with open('transformer_state.pkl', 'rb') as f:
    state = pickle.load(f)

# Initialize transformer with saved state
transformer = SMILESToSELFIESTransformer(pad_to_len=state['pad_to_len'])
transformer.symbol_to_idx = state['symbol_to_idx']
transformer.idx_to_symbol = state['idx_to_symbol']
transformer.alphabet = state['alphabet']

# Transform the new database (df_new)
selfies_features_df = transformer.transform(df)

In [7]:
selfies_features_df.head()

,0,1,2,3,4,5,6,7,8,9,...,165,166,167,168,169,170,171,172,173,174
0,38,12,34,26,10,26,15,26,12,40,...,49,49,49,49,49,49,49,49,49,49
1,38,12,33,26,10,26,15,26,33,26,...,49,49,49,49,49,49,49,49,49,49
2,26,26,12,22,26,26,26,10,26,15,...,49,49,49,49,49,49,49,49,49,49
3,38,12,26,12,26,10,26,15,34,40,...,49,49,49,49,49,49,49,49,49,49
4,30,38,26,10,26,15,26,12,22,0,...,49,49,49,49,49,49,49,49,49,49


In [8]:
# Alphabet list size (0 to 48)
alphabet_size = 49

# Initialize a new DataFrame to store the binary features (columns: 0 to 48)
binary_features_df = pd.DataFrame(0, index=df.index, columns=range(alphabet_size))

# Function to create a binary vector for each row in IDs_All
def create_binary_vector(row, alphabet_size):
    binary_vector = [0] * alphabet_size
    for num_id in row:
        if isinstance(num_id, int) and 0 <= num_id < alphabet_size:  # Ensure only numeric IDs are considered
            binary_vector[num_id] = 1
    return binary_vector


In [9]:
# Apply the function to each row of IDs_All, excluding the 'smiles' column
for idx, row in selfies_features_df.iterrows():
    binary_features_df.loc[idx] = create_binary_vector(row, alphabet_size)

In [10]:
binary_features_df.head()

,0,1,2,3,4,5,6,7,8,9,...,39,40,41,42,43,44,45,46,47,48
0,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,1,0,1,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,1,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,1,1,0,0,0,0,0,0,0


In [11]:
binary_features_df.shape

(18, 49)

In [12]:
# Alphabet list from 0 to 48, corresponding to the numerical IDs
alphabet_list = ['[#Branch1]', '[#Branch2]', '[#C]', '[#N]', '[-\\Ring1]', '[-\\Ring2]', '[/C]', '[/N]', '[/O]', '[/S]', 
                 '[=Branch1]', '[=Branch2]', '[=C]', '[=N+1]', '[=N]', '[=O]', '[=P]', '[=Ring1]', '[=Ring2]', '[=S]',
                 '[B]', '[Br]', '[Branch1]', '[Branch2]', '[C@@H1]', '[C@H1]', '[C]', '[Cl]', '[F]', '[I]', '[Li]', 
                 '[N+1]', '[N-1]', '[NH1]', '[N]', '[Na]', '[O-1]', '[OH0]', '[O]', '[P]', '[Ring1]', '[Ring2]', 
                 '[S+1]', '[S]', '[\\C]', '[\\F]', '[\\N]', '[\\O]', '[\\S]']

# Assign the alphabet_list as the column names for binary_features_df
binary_features_df.columns = alphabet_list


In [13]:
binary_features_df.head()

,[#Branch1],[#Branch2],[#C],[#N],[-\Ring1],[-\Ring2],[/C],[/N],[/O],[/S],...,[P],[Ring1],[Ring2],[S+1],[S],[\C],[\F],[\N],[\O],[\S]
0,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,1,0,1,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,1,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,1,1,0,0,0,0,0,0,0


In [14]:
IDs_All = pd.concat([selfies_features_df, binary_features_df], axis=1)

# Convert all column names to strings
IDs_All.columns = IDs_All.columns.astype(str)

# Now, the column names are all strings and compatible with ML libraries

In [15]:
import pandas as pd
from rdkit import Chem
from rdkit.Chem import Descriptors, rdchem

# Define the smiles_to_features function
def smiles_to_features(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return None
    
    # Count the total number of atoms and bonds in the molecule
    num_atoms = mol.GetNumAtoms()
    num_bonds = mol.GetNumBonds()

    # Check if the molecule contains aromatic rings
    is_molecule_aromatic = any(atom.GetIsAromatic() for atom in mol.GetAtoms() if atom.IsInRing())

    # Check for conjugation
    is_conjugated = False
    for bond in mol.GetBonds():
        if bond.GetBondType() in (rdchem.BondType.DOUBLE, rdchem.BondType.AROMATIC):
            is_conjugated = True
            break

    # Atom features
    atom_features = {
        'avg_hybridization': sum([atom.GetHybridization() for atom in mol.GetAtoms()]) / num_atoms,
        'avg_num_hydrogens': sum([atom.GetTotalNumHs() for atom in mol.GetAtoms()]) / num_atoms
    }
    
    # Additional molecular properties
    molecular_weight = Descriptors.MolWt(mol)  # Molecular weight
    num_rotatable_bonds = Descriptors.NumRotatableBonds(mol)  # Number of rotatable bonds
    num_rings = Descriptors.RingCount(mol)  # Number of rings

    # Count conjugated double bonds (for estimation purposes)
    degree_of_conjugation = sum(1 for bond in mol.GetBonds() if bond.GetBondType() in (rdchem.BondType.DOUBLE, rdchem.BondType.AROMATIC))

    return {
        # Atom features
        **atom_features,
        # Molecular features
        'num_atoms': num_atoms,
        'num_bonds': num_bonds,
        'is_molecule_aromatic': is_molecule_aromatic,
        'is_conjugated': is_conjugated,
        'molecular_weight': molecular_weight,
        'num_rotatable_bonds': num_rotatable_bonds,
        'num_rings': num_rings,
        'degree_of_conjugation': degree_of_conjugation,
    }


In [16]:
# Apply the function to the DataFrame
IDs_All['features'] = df['smiles'].apply(smiles_to_features)

# Convert the 'features' dictionary into separate columns
mol_features_df = pd.json_normalize(IDs_All['features'])

In [17]:
mol_features_df.head()

,avg_hybridization,avg_num_hydrogens,num_atoms,num_bonds,is_molecule_aromatic,is_conjugated,molecular_weight,num_rotatable_bonds,num_rings,degree_of_conjugation
0,3.181818,0.636364,11,12,False,True,153.141,0,2,3
1,3.000000,0.250000,16,18,True,True,220.144,0,3,18
2,3.222222,0.666667,18,19,False,True,248.238,1,2,6
3,3.000000,0.285714,14,15,False,True,192.130,1,2,6
4,2.733333,0.000000,30,31,False,True,391.898,5,2,10


In [18]:
# Convert boolean columns to numerical values (1 for True, 0 for False)
mol_features_df['is_molecule_aromatic'] = mol_features_df['is_molecule_aromatic'].astype(int)
mol_features_df['is_conjugated'] = mol_features_df['is_conjugated'].astype(int)

In [19]:
# Delete the 'is_conjugated' column from features_df
mol_features_df = mol_features_df.drop(columns=['is_conjugated'])

In [20]:
mol_features_df.head()

,avg_hybridization,avg_num_hydrogens,num_atoms,num_bonds,is_molecule_aromatic,molecular_weight,num_rotatable_bonds,num_rings,degree_of_conjugation
0,3.181818,0.636364,11,12,0,153.141,0,2,3
1,3.000000,0.250000,16,18,1,220.144,0,3,18
2,3.222222,0.666667,18,19,0,248.238,1,2,6
3,3.000000,0.285714,14,15,0,192.130,1,2,6
4,2.733333,0.000000,30,31,0,391.898,5,2,10


In [21]:
IDs_All.head()

,0,1,2,3,4,5,6,7,8,9,...,[Ring1],[Ring2],[S+1],[S],[\C],[\F],[\N],[\O],[\S],features
0,38,12,34,26,10,26,15,26,12,40,...,1,0,0,0,0,0,0,0,0,"{'avg_hybridization': 3.1818181818181817, 'avg..."
1,38,12,33,26,10,26,15,26,33,26,...,1,0,0,0,0,0,0,0,0,"{'avg_hybridization': 3.0, 'avg_num_hydrogens'..."
2,26,26,12,22,26,26,26,10,26,15,...,1,0,0,1,0,0,0,0,0,"{'avg_hybridization': 3.2222222222222223, 'avg..."
3,38,12,26,12,26,10,26,15,34,40,...,1,0,0,0,0,0,0,0,0,"{'avg_hybridization': 3.0, 'avg_num_hydrogens'..."
4,30,38,26,10,26,15,26,12,22,0,...,1,1,0,0,0,0,0,0,0,"{'avg_hybridization': 2.7333333333333334, 'avg..."


In [22]:
# Drop the 'features' column if it's no longer needed
IDs_All.drop(columns=['features'], inplace=True)

In [23]:
# Concatenate the original DataFrame with the new features DataFrame
IDs_All = pd.concat([IDs_All, mol_features_df], axis=1)

In [24]:
IDs_All.head()

,0,1,2,3,4,5,6,7,8,9,...,[\S],avg_hybridization,avg_num_hydrogens,num_atoms,num_bonds,is_molecule_aromatic,molecular_weight,num_rotatable_bonds,num_rings,degree_of_conjugation
0,38,12,34,26,10,26,15,26,12,40,...,0,3.181818,0.636364,11,12,0,153.141,0,2,3
1,38,12,33,26,10,26,15,26,33,26,...,0,3.000000,0.250000,16,18,1,220.144,0,3,18
2,26,26,12,22,26,26,26,10,26,15,...,0,3.222222,0.666667,18,19,0,248.238,1,2,6
3,38,12,26,12,26,10,26,15,34,40,...,0,3.000000,0.285714,14,15,0,192.130,1,2,6
4,30,38,26,10,26,15,26,12,22,0,...,0,2.733333,0.000000,30,31,0,391.898,5,2,10


In [25]:
IDs_All.shape

(18, 233)

In [26]:
# Load the trained XGBoost model
model_path = "updated_model_XGBoost_MolFeatures_Oct23.pkl"
with open(model_path, "rb") as f:
    model = pickle.load(f)

In [27]:
# Use the model to make predictions
predictions = model.predict(IDs_All)

# Add predictions as a new column to the dataframe
df["Voltage_Identification_Label"] = predictions



In [28]:
df.head()

,smiles,Voltage_Identification_Label
0,O=C1NC(C2=C1NCCN2)=O,1
1,O=C1NC(C2=C1NC3=C(C(NC3=O)=O)N2)=O,0
2,O=C1N(N2C(C(C)=C(C)C2=O)=O)C(C(C)=C1C)=O,1
3,O=C1N(C(C=C1)=O)N2C(C=CC2=O)=O,0
4,O=C1N(C(C(C(O[Li])=O)=C1C(O[Li])=O)=O)N2C(C(OC...,0


In [29]:

# Save the updated dataframe to a new CSV file
output_csv_path = "candidates_identification_with_labels.csv"
df.to_csv(output_csv_path, index=False)